In [1]:
import json
import requests
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# DATA FROM USDA Food Surveys Research Group: Beltsville, MD

#  LINK = https://www.ars.usda.gov/northeast-area/beltsville-md-bhnrc/beltsville-human-nutrition-research-center/\
# food-surveys-research-group/docs/fndds-download-databases/

# FILE - 2017-2018 FNDDS At A Glance - FNDDS Ingredients 

csv_file = "Resources_usda/usda_17_18_Ingredients.csv"


In [3]:
# MAIN FOOD AVERSION
    # Gluten Free
    # Vegan
    # Vegetarian
    # Ketogenic (this data doesn't have Ketogenic)

ingredients_data_df = pd.read_csv(csv_file)
ingredients_data_df.head()

,Food code,Main food description,WWEIA Category number,WWEIA Category description,Seq num,Ingredient code,Ingredient description,Ingredient weight (g),Retention code,Moisture change (%)
0,11000000,"Milk, human",9602,Human milk,1,1107,"Milk, human, mature, fluid",100.0,0,0.0
1,11100000,"Milk, NFS",1004,"Milk, reduced fat",1,1077,"Milk, whole, 3.25% milkfat, with added vitamin D",35.0,0,0.0
2,11100000,"Milk, NFS",1004,"Milk, reduced fat",2,1079,"Milk, reduced fat, fluid, 2% milkfat, with add...",37.0,0,0.0
3,11100000,"Milk, NFS",1004,"Milk, reduced fat",3,1082,"Milk, lowfat, fluid, 1% milkfat, with added vi...",17.0,0,0.0
4,11100000,"Milk, NFS",1004,"Milk, reduced fat",4,1085,"Milk, nonfat, fluid, with added vitamin A and ...",11.0,0,0.0


In [4]:
ingredients_data_df.shape

(24896, 10)

In [5]:
# Remove rows missing data
ingredients_data_df = ingredients_data_df.dropna()

# Shape total after cleaning 
ingredients_data_df.shape

(24883, 10)

In [6]:
#checking columns and null values
ingredients_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24883 entries, 0 to 24895
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Food code                   24883 non-null  int64  
 1   Main food description       24883 non-null  object 
 2   WWEIA Category number       24883 non-null  int64  
 3   WWEIA Category description  24883 non-null  object 
 4   Seq num                     24883 non-null  int64  
 5   Ingredient code             24883 non-null  int64  
 6   Ingredient description      24883 non-null  object 
 7   Ingredient weight (g)       24883 non-null  float64
 8   Retention code              24883 non-null  int64  
 9   Moisture change (%)         24883 non-null  float64
dtypes: float64(2), int64(5), object(3)
memory usage: 2.1+ MB


In [7]:
ingredients_data_df["WWEIA Category description"].value_counts()

Meat mixed dishes                                   2162
Fish                                                1569
Soups                                               1127
Cakes and pies                                      1062
Pasta mixed dishes, excludes macaroni and cheese    1057
                                                    ... 
Tap water                                              1
Human milk                                             1
Baby water                                             1
Bottled water                                          1
Grapes                                                 1
Name: WWEIA Category description, Length: 167, dtype: int64

In [8]:
# Filtering columns 
reduced_data = ingredients_data_df.loc[:,['Food code', 'Main food description', 'WWEIA Category description', 'Ingredient description']]
reduced_data

,Food code,Main food description,WWEIA Category description,Ingredient description
0,11000000,"Milk, human",Human milk,"Milk, human, mature, fluid"
1,11100000,"Milk, NFS","Milk, reduced fat","Milk, whole, 3.25% milkfat, with added vitamin D"
2,11100000,"Milk, NFS","Milk, reduced fat","Milk, reduced fat, fluid, 2% milkfat, with add..."
3,11100000,"Milk, NFS","Milk, reduced fat","Milk, lowfat, fluid, 1% milkfat, with added vi..."
4,11100000,"Milk, NFS","Milk, reduced fat","Milk, nonfat, fluid, with added vitamin A and ..."
...,...,...,...,...
24891,99998130,Sauce as ingredient in hamburgers,Not included in a food category,"Mustard, prepared, yellow"
24892,99998210,Industrial oil as ingredient in food,Not included in a food category,"Oil, industrial, canola, high oleic"
24893,99998210,Industrial oil as ingredient in food,Not included in a food category,"Oil, industrial, soy, fully hydrogenated"
24894,99998210,Industrial oil as ingredient in food,Not included in a food category,"Oil, industrial, soy, low linolenic"


In [9]:
reduced_data

,Food code,Main food description,WWEIA Category description,Ingredient description
0,11000000,"Milk, human",Human milk,"Milk, human, mature, fluid"
1,11100000,"Milk, NFS","Milk, reduced fat","Milk, whole, 3.25% milkfat, with added vitamin D"
2,11100000,"Milk, NFS","Milk, reduced fat","Milk, reduced fat, fluid, 2% milkfat, with add..."
3,11100000,"Milk, NFS","Milk, reduced fat","Milk, lowfat, fluid, 1% milkfat, with added vi..."
4,11100000,"Milk, NFS","Milk, reduced fat","Milk, nonfat, fluid, with added vitamin A and ..."
...,...,...,...,...
24891,99998130,Sauce as ingredient in hamburgers,Not included in a food category,"Mustard, prepared, yellow"
24892,99998210,Industrial oil as ingredient in food,Not included in a food category,"Oil, industrial, canola, high oleic"
24893,99998210,Industrial oil as ingredient in food,Not included in a food category,"Oil, industrial, soy, fully hydrogenated"
24894,99998210,Industrial oil as ingredient in food,Not included in a food category,"Oil, industrial, soy, low linolenic"


In [10]:
df = reduced_data[reduced_data['Main food description'].str.contains( 'Vegetarian | gluten')]
df.head(100)

,Food code,Main food description,WWEIA Category description,Ingredient description
11808,41811890,"Vegetarian burger or patty, meatless, no bun",Processed soy products,"Veggie burgers or soyburgers, unprepared"
11816,41812400,Vegetarian pot pie,"Bean, pea, legume dishes","Wheat flour, white, all-purpose, enriched, ble..."
11817,41812400,Vegetarian pot pie,"Bean, pea, legume dishes","Shortening, vegetable, household, composite"
11818,41812400,Vegetarian pot pie,"Bean, pea, legume dishes","Margarine, regular, 80% fat, composite, stick,..."
11819,41812400,Vegetarian pot pie,"Bean, pea, legume dishes","Beverages, water, tap, drinking"
...,...,...,...,...
16955,58109130,"Pizza, with meat, gluten-free thick crust",Pizza,"Sausage, Italian, pork, mild, cooked, pan-fried"
16956,58109140,"Pizza, cheese and vegetables, gluten-free thin...",Pizza,"Bread, gluten-free, white, made with rice flou..."
16957,58109140,"Pizza, cheese and vegetables, gluten-free thin...",Pizza,"Sauce, pasta, spaghetti/marinara, ready-to-serve"
16958,58109140,"Pizza, cheese and vegetables, gluten-free thin...",Pizza,"Oil, olive, salad or cooking"


In [11]:
searchfor = ['Veg','veg', 'gluten', 'Gluten']
df= reduced_data[reduced_data['Main food description'].str.contains('|'.join(searchfor))]
df

,Food code,Main food description,WWEIA Category description,Ingredient description
147,11440070,"Vegetable dip, yogurt based","Dips, gravies, other sauces","Salad dressing, mayonnaise, regular"
148,11440070,"Vegetable dip, yogurt based","Dips, gravies, other sauces","Yogurt, Greek, plain, lowfat"
149,11440070,"Vegetable dip, yogurt based","Dips, gravies, other sauces","Salt, table, iodized"
150,11440070,"Vegetable dip, yogurt based","Dips, gravies, other sauces","Onions, cooked, as ingredient"
151,11440070,"Vegetable dip, yogurt based","Dips, gravies, other sauces","Spices, parsley, dried"
...,...,...,...,...
24884,99997810,Vegetables as ingredient in curry,Not included in a food category,"Carrots, cooked, as ingredient"
24885,99997810,Vegetables as ingredient in curry,Not included in a food category,"Cauliflower, cooked, as ingredient"
24886,99997810,Vegetables as ingredient in curry,Not included in a food category,"Peas, green, frozen, cooked, boiled, drained, ..."
24887,99997810,Vegetables as ingredient in curry,Not included in a food category,"Green beans, cooked, as ingredient"


In [12]:
df['Food code'].nunique()

646

In [13]:
df['Ingredient description']= df['Ingredient description'].str.replace(',',' ')

<ipython-input-13-c4b391a59473>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Ingredient description']= df['Ingredient description'].str.replace(',',' ')


In [14]:
df


,Food code,Main food description,WWEIA Category description,Ingredient description
147,11440070,"Vegetable dip, yogurt based","Dips, gravies, other sauces",Salad dressing mayonnaise regular
148,11440070,"Vegetable dip, yogurt based","Dips, gravies, other sauces",Yogurt Greek plain lowfat
149,11440070,"Vegetable dip, yogurt based","Dips, gravies, other sauces",Salt table iodized
150,11440070,"Vegetable dip, yogurt based","Dips, gravies, other sauces",Onions cooked as ingredient
151,11440070,"Vegetable dip, yogurt based","Dips, gravies, other sauces",Spices parsley dried
...,...,...,...,...
24884,99997810,Vegetables as ingredient in curry,Not included in a food category,Carrots cooked as ingredient
24885,99997810,Vegetables as ingredient in curry,Not included in a food category,Cauliflower cooked as ingredient
24886,99997810,Vegetables as ingredient in curry,Not included in a food category,Peas green frozen cooked boiled drained ...
24887,99997810,Vegetables as ingredient in curry,Not included in a food category,Green beans cooked as ingredient


In [15]:
df.groupby('Food code')['Ingredient description'].apply(', '.join)

Food code
11440070    Salad dressing  mayonnaise  regular, Yogurt  G...
12350250    Salad dressing  mayonnaise  regular, Cream  so...
12350255    Salad dressing  mayonnaise  light, Sour cream ...
14202020    Cheese  cottage  lowfat  2% milkfat, Vegetable...
14410600    Cheese  cheddar (Includes foods for USDA's Foo...
                                  ...                        
92550400    Beverages  Vegetable and fruit juice drink  re...
92550405    Beverages  Vegetable and fruit juice drink  re...
99997800    Spinach  cooked  as ingredient, Broccoli  cook...
99997804    Mushrooms  cooked  as ingredient, Onions  cook...
99997810    Potatoes  baked  flesh and skin  without salt,...
Name: Ingredient description, Length: 646, dtype: object

In [16]:
df_merge_col = df.loc[~ df[['Food code', 'Main food description', 'WWEIA Category description']].duplicated(),['Food code', 'Main food description', 'WWEIA Category description']]
df_merge_col.set_index('Food code', inplace= True)
df_merge_col

,Main food description,WWEIA Category description
Food code,,
11440070,"Vegetable dip, yogurt based","Dips, gravies, other sauces"
12350250,"Vegetable dip, regular","Dips, gravies, other sauces"
12350255,"Vegetable dip, light","Dips, gravies, other sauces"
14202020,"Cheese, cottage, with vegetables",Cottage/ricotta cheese
14410600,"Cheese, processed, with vegetables",Cheese
...,...,...
92550400,"Vegetable and fruit juice drink, with high vit...",Other diet drinks
92550405,"Vegetable and fruit juice drink, with high vit...",Fruit drinks
99997800,Dark green vegetables as ingredient in omelet,Not included in a food category


In [17]:
df_merge_col['Recipe']= df.groupby('Food code')['Ingredient description'].apply(', '.join)
df_merge_col

,Main food description,WWEIA Category description,Recipe
Food code,,,
11440070,"Vegetable dip, yogurt based","Dips, gravies, other sauces","Salad dressing mayonnaise regular, Yogurt G..."
12350250,"Vegetable dip, regular","Dips, gravies, other sauces","Salad dressing mayonnaise regular, Cream so..."
12350255,"Vegetable dip, light","Dips, gravies, other sauces","Salad dressing mayonnaise light, Sour cream ..."
14202020,"Cheese, cottage, with vegetables",Cottage/ricotta cheese,"Cheese cottage lowfat 2% milkfat, Vegetable..."
14410600,"Cheese, processed, with vegetables",Cheese,Cheese cheddar (Includes foods for USDA's Foo...
...,...,...,...
92550400,"Vegetable and fruit juice drink, with high vit...",Other diet drinks,Beverages Vegetable and fruit juice drink re...
92550405,"Vegetable and fruit juice drink, with high vit...",Fruit drinks,Beverages Vegetable and fruit juice drink re...
99997800,Dark green vegetables as ingredient in omelet,Not included in a food category,"Spinach cooked as ingredient, Broccoli cook..."


In [18]:
df_merge_col

,Main food description,WWEIA Category description,Recipe
Food code,,,
11440070,"Vegetable dip, yogurt based","Dips, gravies, other sauces","Salad dressing mayonnaise regular, Yogurt G..."
12350250,"Vegetable dip, regular","Dips, gravies, other sauces","Salad dressing mayonnaise regular, Cream so..."
12350255,"Vegetable dip, light","Dips, gravies, other sauces","Salad dressing mayonnaise light, Sour cream ..."
14202020,"Cheese, cottage, with vegetables",Cottage/ricotta cheese,"Cheese cottage lowfat 2% milkfat, Vegetable..."
14410600,"Cheese, processed, with vegetables",Cheese,Cheese cheddar (Includes foods for USDA's Foo...
...,...,...,...
92550400,"Vegetable and fruit juice drink, with high vit...",Other diet drinks,Beverages Vegetable and fruit juice drink re...
92550405,"Vegetable and fruit juice drink, with high vit...",Fruit drinks,Beverages Vegetable and fruit juice drink re...
99997800,Dark green vegetables as ingredient in omelet,Not included in a food category,"Spinach cooked as ingredient, Broccoli cook..."


In [28]:
df_merge_col = df_merge_col.reset_index()
df_merge_col

,Food code,Main food description,WWEIA Category description,Recipe
0,11440070,"Vegetable dip, yogurt based","Dips, gravies, other sauces","Salad dressing mayonnaise regular, Yogurt G..."
1,12350250,"Vegetable dip, regular","Dips, gravies, other sauces","Salad dressing mayonnaise regular, Cream so..."
2,12350255,"Vegetable dip, light","Dips, gravies, other sauces","Salad dressing mayonnaise light, Sour cream ..."
3,14202020,"Cheese, cottage, with vegetables",Cottage/ricotta cheese,"Cheese cottage lowfat 2% milkfat, Vegetable..."
4,14410600,"Cheese, processed, with vegetables",Cheese,Cheese cheddar (Includes foods for USDA's Foo...
...,...,...,...,...
641,92550400,"Vegetable and fruit juice drink, with high vit...",Other diet drinks,Beverages Vegetable and fruit juice drink re...
642,92550405,"Vegetable and fruit juice drink, with high vit...",Fruit drinks,Beverages Vegetable and fruit juice drink re...
643,99997800,Dark green vegetables as ingredient in omelet,Not included in a food category,"Spinach cooked as ingredient, Broccoli cook..."
644,99997804,Other vegetables as ingredient in omelet,Not included in a food category,"Mushrooms cooked as ingredient, Onions cook..."


In [21]:
df_merge_col.to_csv('Resources_usda/merge_recipe.csv')

In [22]:
df.to_csv('Resources_usda/food_adversion.csv')

In [23]:
!pip install psycopg2

In [29]:
connection_string = "postgres:postgres@localhost:5432/recipe_adversion"
engine = create_engine(f'postgresql://{connection_string}')

In [30]:
engine.table_names()

<ipython-input-30-d72bc75a6b62>:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['recipe_adversion']

In [31]:
df_merge_col.to_sql(name='recipe_adversion', con=engine, if_exists='append', index=False)


In [32]:
pd.read_sql_query('select * from recipe_adversion', con=engine).head()

,Food code,Main food description,WWEIA Category description,Recipe
0,11440070,"Vegetable dip, yogurt based","Dips, gravies, other sauces","Salad dressing mayonnaise regular, Yogurt G..."
1,12350250,"Vegetable dip, regular","Dips, gravies, other sauces","Salad dressing mayonnaise regular, Cream so..."
2,12350255,"Vegetable dip, light","Dips, gravies, other sauces","Salad dressing mayonnaise light, Sour cream ..."
3,14202020,"Cheese, cottage, with vegetables",Cottage/ricotta cheese,"Cheese cottage lowfat 2% milkfat, Vegetable..."
4,14410600,"Cheese, processed, with vegetables",Cheese,Cheese cheddar (Includes foods for USDA's Foo...


In [ ]:
### The end ### 